# About this notebook 

Implementing and testing the statistical parity with the political blog network. 

Source:
- [Fairwalk paper](https://www.ijcai.org/proceedings/2019/456)

## Statistical parity 

Statistical parity is a variance of the probability of edges at group level. The key idea is the following. Let's group nodes based on given node labels. If the presence of edges are independent of the given node labels, the probability of edges within and between the groups are the uniform. More specifically, suppose a network with edges recommended by a system. Assuming that the network is symmetric, 
the probability of the recommended edges between group $k$ and group $\ell$, denoted by Let $P_{k\ell}$, is given by 
$$
\begin{align}
P_{k\ell} = \left\{ \begin{array}{cc} 
\frac{1}{N_{k}(N_{k} - 1)/2} \sum_{i \in N_k}\sum_{j \in N_k, i<j} A_{ij}& \text{(if $k=\ell$)}\\
\frac{1}{N_{k}N_{\ell}} \sum_{i \in N_k}\sum_{i \in N_\ell}A_{ij} & \text{(otherwise)}\\
\end{array} \right.
\end{align}
$$
where $A_{ij} = 1$ if nodes $i$ and $j$ are connected, otherwise $A_{ij}=0$. 
The denominator $N_k(N_{k} - 1)/2$ is the total number of unique node pairs that belong to group $k$, excluding the self-loops, e.g., $(1, 1)$, $(2,2), \ldots, (N_k, N_k)$.  
Then, the statistical parity is given by the variance of the probabilities:
$$
\begin{align}
\text{Var}(P_{k \ell}) = \frac{1}{K(K+1)/2}\sum_{k=1}^K \sum_{\ell=k}^K P_{k\ell}.
\end{align}
$$

While the definition of the statistical parity in the fairwalk paper is variance, the original definition of the statistical parity is the *standard deviation*. I believe that this is an error. The correct formula for the statistical parity is the following:
$$
\begin{align}
\sqrt{\text{Var}(P_{k \ell})} = \sqrt{\frac{1}{K(K+1)/2}\sum_{k=1}^K \sum_{\ell=k}^K P_{k\ell}}.
\end{align}
$$


## Implementation 

In [19]:
from scipy import sparse 
import numpy as np 
import pandas as pd 

def statistical_parity(edges, y):
    """
    edges: edge df with source and target column
    y: original labels
    """

    n_nodes = len(y) # number of nodes
    n_edges = edges.shape[0] # number of nodes
    uy, y = np.unique(y, return_inverse=True) # to ensure that the labels are continuous integers starting from zero 
    K = len(uy) # number of classes

    # We need two groups at least
    assert K >= 2

    # Group membership matrix, where U[i,k] = 1 if node $i$ belongs to group $k$ 
    U = sparse.csr_matrix((np.ones_like(y), (np.arange(n_nodes), y)), shape=(n_nodes, K))

    # Number of nodes in each group
    Nk = np.array(U.sum(axis = 0)).reshape(-1) 

    # Adjacency matrix
    A = sparse.csr_matrix((np.ones(n_edges), (edges["source"].values, edges["target"].values)), shape=(n_nodes, n_nodes))

    # Make sure that the adjacency matrix is symemtric 
    A = A + A.T
    A.data = A.data *0 + 1

    # Calculate the number of edges that appear in each group 
    M = U.T @ A @ U

    # Calculate the edge density
    Mdenom = np.outer(Nk, Nk) - np.diag(Nk) 
    P = M / Mdenom
    # Calculate the statistical parity 
    parity = np.std(P[np.triu_indices(K)])
    return parity

## Test

In the following test, I calculate the statistical parity for the original network. As a baseline, I calculate the statistical parity for shuffled labels, which should have a smaller statistical parity. 

In [32]:
import networkx as nx
 
G = nx.read_gml("../../data/polbooks.gml")
labels = [d[1]["value"] for d in G.nodes(data = True)]

source, target, _ = sparse.find(nx.adjacency_matrix(G))
edges = pd.DataFrame({"source":source, "target":target})

random_labels = np.random.choice(len(set(labels)), len(labels), replace=True)

score = statistical_parity(edges, labels)
score_random = statistical_parity(edges, random_labels) 
print(f"Statistical parity: original labels= {score:.4f}, random labels = {score_random:.4f}")

Statistical parity: original labels= 0.0663, random labels = 0.0185
